In [7]:
import numpy as np
import pandas as pd
import pymongo

In [29]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, run = False, credentials = "credentials.txt", db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials = "credentials.txt"):
        try:
            #use your database name, user and password here:
            with open(credentials, 'r', encoding='utf-8') as f:
                [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection Update in Local.")
        return None
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if name_collection in self.collections:
            return self.collections[name_collection]
        return None
        
MS = MongoServer(True)
MS.listOfCollections()
MS.getCollectionFromServer('ingredients')

Collection(Database(MongoClient(host=['ds233895.mlab.com:33895'], document_class=dict, tz_aware=False, connect=True), 'agile_data_science_group_3'), 'ingredients')

In [26]:
R = Recommender()
db = R.getDataBase()
R.getCollection(db, "recipes")

{'_id': ObjectId('59fb0d77eee3642f3c0683f7'), 'id': '20-minute-carbonara-with-peas', 'url': 'https://www.asdagoodliving.co.uk/food/recipes/20-minute-carbonara-with-peas', 'image': 'https://be35832fa5168a30acd6-5c7e0f2623ae37b4a933167fe83d71b5.ssl.cf3.rackcdn.com/1602/20-minute-carbonara-with-peas__square.jpg', 'name': '20-minute carbonara with peas', 'serves': '4', 'energy_KCal': 431, 'salt_mg': '1805', 'saturated_fat_g': '6.1', 'fat_g': '16.6', 'sugar_g': '5', 'time_min': '20', 'price_p': '115', 'list_ingredients': ['200g spaghetti', '250g frozen peas', '2 medium eggs', '25g Parmesan(grated)', '4 level tbsp reduced-fat crème fraîche', '1 tbsp olive oil', '4 rashers Asda Extra Special Unsmoked Back Bacon(chopped)', '1 shallot(finely chopped)', '1 garlic clove(crushed)', 'Chopped parsley(to serve)'], 'ningredients': 10.0}


In [ ]:
print(db.collection_names())
recipes = db.recipes
print()
print(recipes)

In [ ]:
recipes.count()

In [ ]:
for ing in recipes.find():
    print(ing)

In [ ]:
class Recommender:
    
    def __init__(self):
        pass
